In [ ]:
# # no need for this since it has nothing outside classes and functions
# print(__name__)
# if __name__ == "__main__" and hasattr(__builtins__,'__IPYTHON__') and ('google.colab' in str(get_ipython())):
#     from google.colab import drive
#     drive.mount('/content/drive')
#     %cd /content/drive/MyDrive/PressureReliefWorkArea/SummerWork/
#     !ls

__main__
Mounted at /content/drive
/content/drive/MyDrive/PressureReliefWorkArea/SummerWork
AbstractModel.ipynb  DataProcessing.ipynb   RunAndEvaluation.ipynb
CNNModel.ipynb	     HelperFunctions.ipynb


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%run -n AbstractModel.ipynb

In this code, we added 2 more convolutional layers, which can extract more complex features from your accelerometer data. The number of output channels in the convolutional layers gradually increases, as it is common in many deep learning models to gradually increase the complexity and decrease the spatial size.

Please note that the dimension of the input to the fully connected layer depends on the output size of your last convolutional layer. This code assumes that after 3 layers of convolution with kernel size 5 and stride 1, the sequence length is reduced to 82 (from the original 100). You may need to adjust this according to your own situation.

In [ ]:
class Net(nn.Module):
    def __init__(self, class_type, sequence_length):
        super(Net, self).__init__()
        # the actual net parts were moved into the following function
            # since we need additional information before setup

    # def init_net(self):
        input_num = SKDescriptors.NUM_OF_INPUTS_PER_TYPE[class_type]
        class_num = SKDescriptors.NUM_OF_CLASSES_PER_TYPE[class_type]

        self.conv1 = nn.Conv1d(input_num, 64, kernel_size=3)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3)

        # Adjust the fully connected layer's input size based on the new sequence length after convolutions.
        # Adjusted for sequence length = 4 after 3 conv layers with kernel size 3
        # 10 -3 + 1 = 8 after the first layer
        # 8 - 3 + 1 = 6 after the second layer
        # 6 - 3 + 1 = 4 after the third layer
        self.fc1 = nn.Linear(256 * (sequence_length - 6), 128)  # Adjusted for sequence length = 4 after 3 conv layers with kernel size 3
        self.fc2 = nn.Linear(128, class_num)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.num_flat_features(x))  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # Apply softmax to the output layer

    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
class JFNetModel(SKJFModel):
    def __init__(self, sequence_length = 10):
        # this should get sent to SKJFModel first, which needs sequence_length,
            # and which then calls super().__init__() with no arguments,
            # which should be sent to nn.Module's __init__()
        super(JFNetModel, self).__init__(sequence_length)
        # self.sequence_length = sequence_length
        self.net = None


    def get_outputs(self, inputs, feedback = False):
        return self.net(inputs)


    # def skm_load_data(self, file_path):
    #     # super(JFSKNetModel, self).skm_load_data(file_path)
    #     SKJFModel.skm_load_data(self, file_path)
    
    def load_model(self, file_path = None, class_type = None):
        if class_type is not None:
            self.train_class_type = class_type
        if file_path is None:
            self.net = Net(self.train_class_type, self.sequence_length)
            return
        
        match SKFileNameHandler.read_file_extension(file_path):
            case '.pth':
                self.net = torch.load(file_path)
            case _:
                raise NotImplementedError(f"JFNetModel.load_model() is not equipped to open {_} files.")


# still working on this
    def train(self, save_file_path, epochs = 1000, feedback = False):
        # Instantiate the network and optimizer
        if self.net is None:
            self.load_model()
        optimizer = optim.SGD(self.net.parameters(), lr=0.01)

        # Define the loss function
        criterion = nn.CrossEntropyLoss()

        # Assume we have a data loader `train_dataloader` which loads our training accelerometer data
        for epoch in range(epochs + 1):  # loop over the dataset multiple times

            running_loss = 0.0
            # for i, data in enumerate(self.get_loader().select("Train"), 0): # enumerate(self.train_loader.train_dataloader, 0):
            #     print(i)
            #     print(data)
            for i, data in enumerate(self.get_loader().select("Train"), 0): # enumerate(self.train_loader.train_dataloader, 0):
            # for i, *args in enumerate(self.get_loader().select("Train"), 0): # enumerate(self.train_loader.train_dataloader, 0):
            #     # get the inputs; data is a list of [inputs, labels]
            #     print(args)
            #     if len(args) != 1:
            #         print(len(args))
            #         raise Exception("yippee")
            #     data = args[0]
                inputs, labels = data

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self.net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if (epoch % 200) == 0:    # print every 2000 mini-batches
                    if feedback or not i:
                        print('[%d, %5d] loss: %.3f' %
                            (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0

        print('Finished Training')

        # Now we will validate the model using test data
        correct = 0
        total = 0

        with torch.no_grad():   # Since we're not training, we don't need to calculate the gradients
            for data in self.get_loader().select("Test"): # self.train_loader.test_dataloader:
                inputs, labels = data
                outputs = self.net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))


        # Saving the entire model

        torch.save(self.net, save_file_path)